In [ ]:
!pip install -q openai
!pip install -q datasets==2.12.0

Downloading dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
from datasets import load_dataset
import pickle
import pandas as pd

In [ ]:
dataset = load_dataset("AmazonScience/massive", name='fa-IR')
dataset = dataset.shuffle(seed=42)

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'locale', 'partition', 'scenario', 'intent', 'utt', 'annot_utt', 'worker_id', 'slot_method', 'judgments'],
        num_rows: 11514
    })
    validation: Dataset({
        features: ['id', 'locale', 'partition', 'scenario', 'intent', 'utt', 'annot_utt', 'worker_id', 'slot_method', 'judgments'],
        num_rows: 2033
    })
    test: Dataset({
        features: ['id', 'locale', 'partition', 'scenario', 'intent', 'utt', 'annot_utt', 'worker_id', 'slot_method', 'judgments'],
        num_rows: 2974
    })
})

In [ ]:
test = pd.DataFrame(dataset['test'])
test['intent']

0       48
1       46
2        1
3       41
4       40
        ..
2969    33
2970    44
2971    44
2972    44
2973    44
Name: intent, Length: 2974, dtype: int64

In [ ]:
test.head()

,id,locale,partition,scenario,intent,utt,annot_utt,worker_id,slot_method,judgments
0,0,fa-IR,test,16,48,این هفته ساعت پنج صبح بیدارم کن,[date : این هفته] ساعت [time : پنج صبح] بیدارم کن,8,"{'slot': ['time', 'date'], 'method': ['transla...","{'worker_id': ['3', '21', '8'], 'intent_score'..."
1,3,fa-IR,test,10,46,ساکت,ساکت,17,"{'slot': [], 'method': []}","{'worker_id': ['3', '8', '21'], 'intent_score'..."
2,8,fa-IR,test,8,1,صورتی همان چیزی است که نیاز داریم,[color_type : صورتی] همان چیزی است که نیاز داریم,3,"{'slot': ['color_type'], 'method': ['translati...","{'worker_id': ['3', '21', '8'], 'intent_score'..."
3,14,fa-IR,test,8,41,و تاریک شده است,و تاریک شده است,5,"{'slot': [], 'method': []}","{'worker_id': ['21', '14', '3'], 'intent_score..."
4,19,fa-IR,test,8,40,علی چراغ‌های اتاق خواب را خاموش کن,علی چراغ‌های [house_place : اتاق] خواب را خامو...,3,"{'slot': ['house_place'], 'method': ['translat...","{'worker_id': ['8', '14', '3'], 'intent_score'..."


In [ ]:
for i in range(18):
    print(i)
    print(len(test[test['scenario'] == i]))
    print(test[test['scenario'] == i]['intent'].value_counts())
    print('-----------------------------------------------------------')

0
106
47    81
27    25
Name: intent, dtype: int64
-----------------------------------------------------------
1
124
42    51
2     35
54    23
11    15
Name: intent, dtype: int64
-----------------------------------------------------------
2
402
50    209
32    126
30     67
Name: intent, dtype: int64
-----------------------------------------------------------
3
387
45    176
36     72
58     63
20     41
51     35
Name: intent, dtype: int64
-----------------------------------------------------------
4
124
22    124
Name: intent, dtype: int64
-----------------------------------------------------------
5
103
0     88
38    15
Name: intent, dtype: int64
-----------------------------------------------------------
6
94
6     43
19    31
55    20
Name: intent, dtype: int64
-----------------------------------------------------------
7
271
44    119
33    114
17     26
15     12
Name: intent, dtype: int64
-----------------------------------------------------------
8
220
40    43
1     36
56  

In [ ]:
test = test[test['scenario'] == 3] #play
test['intent'].value_counts()

45    176
36     72
58     63
20     41
51     35
Name: intent, dtype: int64

In [ ]:
intent_map = {45: 'play_music', 36: 'play_radio', 58:'play_podcasts', 20:'play_audiobook', 51:'play_game'}

In [ ]:
intent_map

{45: 'play_music',
 36: 'play_radio',
 58: 'play_podcasts',
 20: 'play_audiobook',
 51: 'play_game'}

OpenAI prompting

In [ ]:
import openai
openai.api_key  = ""

def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0,
    )
    return response.choices[0].message["content"]

def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature
    )
    return response.choices[0].message["content"]

In [ ]:
import time

max_retries = 5

In [ ]:
n_test_sample = 1
test_prompt = test.groupby('intent').apply(lambda x: x.sample(n_test_sample))

test_prompt.head()

,,id,locale,partition,scenario,intent,utt,annot_utt,worker_id,slot_method,judgments
intent,,,,,,,,,,,
20,1703,9856,fa-IR,test,3,20,کتاب دا را پخش کن,کتاب [audiobook_name : دا] را پخش کن,3,"{'slot': ['audiobook_name'], 'method': ['local...","{'worker_id': ['21', '3', '8'], 'intent_score'..."
36,1608,9275,fa-IR,test,3,36,فقط رادیو جوان باید از رادیو پخش بشه,فقط [radio_name : رادیو جوان] باید از رادیو پخ...,8,"{'slot': ['radio_name'], 'method': ['localizat...","{'worker_id': ['8', '14', '21'], 'intent_score..."
45,1686,9738,fa-IR,test,3,45,میشه لطفا آهنگ کلاسیک برای من پخش کنی,میشه لطفا آهنگ [music_genre : کلاسیک] برای من ...,3,"{'slot': ['music_genre'], 'method': ['translat...","{'worker_id': ['21', '3', '8'], 'intent_score'..."
51,1798,10319,fa-IR,test,3,51,با من فوتبال بازی کن,با من [game_name : فوتبال] بازی کن,3,"{'slot': ['game_name'], 'method': ['translatio...","{'worker_id': ['14', '3', '8'], 'intent_score'..."
58,1978,11328,fa-IR,test,3,58,فایل صوتی مونولوگ را پخش کن,فایل صوتی [podcast_name : مونولوگ] را پخش کن,3,"{'slot': ['podcast_name'], 'method': ['localiz...","{'worker_id': ['8', '14', '21'], 'intent_score..."


In [ ]:
prompt = f"""
The below sentence is a person's utterance to a virtual assistant. The utterance is in Persian. Identify the intent associated with it. \
Possible answers are: play_music, play_radio, play_podcasts, play_audiobook, play_game. (Please only reply one of these and nothing else!) \

utterance: SAMPLE
intent:
"""

for index, row in test_prompt.iterrows():
    retries = 0
    while retries < max_retries:
        try:
            response = get_completion(prompt.replace("SAMPLE", row['utt']))
            print(f"utterance: {row['utt']}")
            print(f"Model Response: {response}")
            print(f"Actual Label: {intent_map[row['intent']]}")
            print("---------------------------------------------------------------------")
            break
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            retries += 1
            print(f"Retrying ({retries}/{max_retries})...")
            time.sleep(30)

    if retries == max_retries:
        print(f"Failed to process sample after {max_retries} retries. Skipping to the next sample.")


utterance: کتاب دا را پخش کن
Model Response: play_audiobook
Actual Label: play_audiobook
---------------------------------------------------------------------
utterance: فقط رادیو جوان باید از رادیو پخش بشه
Model Response: play_radio
Actual Label: play_radio
---------------------------------------------------------------------
utterance: میشه لطفا آهنگ کلاسیک برای من پخش کنی
Model Response: play_music
Actual Label: play_music
---------------------------------------------------------------------
An error occurred: Rate limit reached for default-gpt-3.5-turbo in organization org-4mvcYVBwmYmrtxYauFgReUJz on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method.
Retrying (1/5)...
utterance: با من فوتبال بازی کن
Model Response: play_game
Actual Label: play_

In [ ]:
acceptable_classes = [
    "play_music",
    "play_radio",
    "play_podcasts",
    "play_audiobook",
    "play_game",
]

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from collections import Counter

def calculate_metrics(actual, predicted):
    accuracy = accuracy_score(actual, predicted)
    f1 = f1_score(actual, predicted, average='weighted', zero_division=1)
    precision = precision_score(actual, predicted, average='weighted', zero_division=1)
    recall = recall_score(actual, predicted, average='weighted', zero_division=1)

    actual_counts = Counter(actual)
    predicted_counts = Counter(predicted)
    classes = set(actual + predicted)

    metrics = {
        'Accuracy': accuracy,
        'F1 Score': f1,
        'Precision': precision,
        'Recall': recall,
        'Class-wise Counts': {}
    }

    for class_label in classes:
        class_precision = precision_score(actual, predicted, labels=[class_label], average='micro', zero_division=1)
        class_recall = recall_score(actual, predicted, labels=[class_label], average='micro', zero_division=1)
        class_f1 = f1_score(actual, predicted, labels=[class_label], average='micro', zero_division=1)

        metrics['Class-wise Counts'][class_label] = {
            'Actual Count': actual_counts[class_label],
            'Predicted Count': predicted_counts[class_label],
            'Precision': class_precision,
            'Recall': class_recall,
            'F1 Score': class_f1
        }

    for metric, value in metrics.items():
        if metric == 'Class-wise Counts':
            print(f"\n{metric}:")
            for class_label, stats in value.items():
                print(f"Class: {class_label}")
                for stat, val in stats.items():
                    print(f"{stat}: {val}")
                print()
        else:
            print(f"{metric}: {value}")

    return metrics

In [ ]:
from tqdm import tqdm

actual = []
predicted = []

prompt = f"""
The below sentence is a person's utterance to a virtual assistant. The utterance is in Persian. Identify the intent associated with it. \
Possible answers are: play_music, play_radio, play_podcasts, play_audiobook, play_game. (Please only reply one of these and nothing else!) \

utterance: SAMPLE
intent:
"""
c = 0
for index, row in tqdm(test.iterrows(), total=len(test)):
    # c += 1
    # if (c == 2):
    #     time.sleep(30)
    #     c = 0
    retries = 0
    while retries < 1: #max_retries:
        # time.sleep(20)
        try:
            response = get_completion(prompt.replace("SAMPLE", row['utt']))
            actual.append(intent_map[row['intent']])
            if response not in acceptable_classes:
                print(prompt, response)

            predicted.append(response)

            break
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            retries += 1
            print(f"Retrying ({retries}/{max_retries})...")
            time.sleep(30)

    time.sleep(20)

    if retries == max_retries:
        print(f"Failed to process sample after {max_retries} retries. Skipping to the next sample.")

  4%|▍         | 15/387 [05:08<2:07:13, 20.52s/it]


The below sentence is a person's utterance to a virtual assistant. The utterance is in Persian. Identify the intent associated with it. Possible answers are: play_music, play_radio, play_podcasts, play_audiobook, play_game. (Please only reply one of these and nothing else!) 
utterance: SAMPLE
intent:
 None of the above (This is not a valid intent for a virtual assistant)


 40%|████      | 156/387 [53:13<1:18:54, 20.50s/it]


The below sentence is a person's utterance to a virtual assistant. The utterance is in Persian. Identify the intent associated with it. Possible answers are: play_music, play_radio, play_podcasts, play_audiobook, play_game. (Please only reply one of these and nothing else!) 
utterance: SAMPLE
intent:
 play_previous_song


 65%|██████▍   | 250/387 [1:25:18<46:46, 20.49s/it]

...

 68%|██████▊   | 264/387 [1:30:05<42:04, 20.53s/it]


The below sentence is a person's utterance to a virtual assistant. The utterance is in Persian. Identify the intent associated with it. Possible answers are: play_music, play_radio, play_podcasts, play_audiobook, play_game. (Please only reply one of these and nothing else!) 
utterance: SAMPLE
intent:
 None of the above (The utterance translates to "Stop" and does not indicate any of the listed intents.)


 86%|████████▋ | 334/387 [1:54:01<18:06, 20.51s/it]


The below sentence is a person's utterance to a virtual assistant. The utterance is in Persian. Identify the intent associated with it. Possible answers are: play_music, play_radio, play_podcasts, play_audiobook, play_game. (Please only reply one of these and nothing else!) 
utterance: SAMPLE
intent:
 upload_audiobooks


100%|██████████| 387/387 [2:12:08<00:00, 20.49s/it]


In [ ]:
actual

['play_music',
 'play_music',
 'play_music',
 'play_music',
 'play_music',
 'play_music',
 'play_music',
 'play_music',
 'play_music',
 'play_music',
 'play_music',
 'play_music',
 'play_music',
 'play_music',
 'play_music',
 'play_music',
 'play_music',
 'play_music',
 'play_music',
 'play_music',
 'play_music',
 'play_music',
 'play_music',
 'play_music',
 'play_music',
 'play_music',
 'play_music',
 'play_music',
 'play_music',
 'play_music',
 'play_music',
 'play_music',
 'play_music',
 'play_music',
 'play_music',
 'play_music',
 'play_music',
 'play_music',
 'play_music',
 'play_music',
 'play_music',
 'play_music',
 'play_music',
 'play_music',
 'play_music',
 'play_music',
 'play_music',
 'play_music',
 'play_music',
 'play_music',
 'play_music',
 'play_music',
 'play_music',
 'play_music',
 'play_music',
 'play_music',
 'play_music',
 'play_music',
 'play_music',
 'play_music',
 'play_music',
 'play_music',
 'play_music',
 'play_music',
 'play_music',
 'play_music',
 'play_mus

In [ ]:
calculate_metrics(actual, predicted)

Accuracy: 0.7700258397932817
F1 Score: 0.7517118450141267
Precision: 0.7925609543559198
Recall: 0.7700258397932817

Class-wise Counts:
Class: None of the above (The utterance translates to "Stop" and does not indicate any of the listed intents.)
Actual Count: 0
Predicted Count: 1
Precision: 0.0
Recall: 1.0
F1 Score: 0.0

Class: None of the above (This is not a valid intent for a virtual assistant)
Actual Count: 0
Predicted Count: 1
Precision: 0.0
Recall: 1.0
F1 Score: 0.0

Class: stop_playing
Actual Count: 0
Predicted Count: 1
Precision: 0.0
Recall: 1.0
F1 Score: 0.0

Class: stop_audiobook
Actual Count: 0
Predicted Count: 1
Precision: 0.0
Recall: 1.0
F1 Score: 0.0

Class: play_music
Actual Count: 176
Predicted Count: 189
Precision: 0.8835978835978836
Recall: 0.9488636363636364
F1 Score: 0.915068493150685

Class: play_podcasts
Actual Count: 63
Predicted Count: 15
Precision: 0.6666666666666666
Recall: 0.15873015873015872
F1 Score: 0.2564102564102564

Class: play_game
Actual Count: 35
Pre

{'Accuracy': 0.7700258397932817,
 'F1 Score': 0.7517118450141267,
 'Precision': 0.7925609543559198,
 'Recall': 0.7700258397932817,
 'Class-wise Counts': {'None of the above (The utterance translates to "Stop" and does not indicate any of the listed intents.)': {'Actual Count': 0,
   'Predicted Count': 1,
   'Precision': 0.0,
   'Recall': 1.0,
   'F1 Score': 0.0},
  'None of the above (This is not a valid intent for a virtual assistant)': {'Actual Count': 0,
   'Predicted Count': 1,
   'Precision': 0.0,
   'Recall': 1.0,
   'F1 Score': 0.0},
  'stop_playing': {'Actual Count': 0,
   'Predicted Count': 1,
   'Precision': 0.0,
   'Recall': 1.0,
   'F1 Score': 0.0},
  'stop_audiobook': {'Actual Count': 0,
   'Predicted Count': 1,
   'Precision': 0.0,
   'Recall': 1.0,
   'F1 Score': 0.0},
  'play_music': {'Actual Count': 176,
   'Predicted Count': 189,
   'Precision': 0.8835978835978836,
   'Recall': 0.9488636363636364,
   'F1 Score': 0.915068493150685},
  'play_podcasts': {'Actual Count': 

In [ ]:
with open(r"/content/drive/Shareddrives/Gdrive/NLP Bachelors' Project/actual.pickle", "wb") as output_file:
    pickle.dump(actual, output_file)

In [ ]:
with open(r"/content/drive/Shareddrives/Gdrive/NLP Bachelors' Project/pred.pickle", "wb") as output_file:
    pickle.dump(predicted, output_file)